# New Schemas/Tables

From an existing data source, create new schemas/tables and insert the new data

When creating new schemas and tables we need to `commit` changes with `psycopg2` connections

In [1]:
import psycopg2
import pandas as pd
from aws_secrets import get_secret

In [2]:
def create_rw_conn(secrets):
    # pass along secrets to pyscopg2
    ENDPOINT = secrets['ENDPOINT']
    PORT = secrets['PORT']
    USER = secrets['USER']
    PASSWORD = secrets['PASSWORD']
    DATABASE = secrets['DATABASE']
    
    # create connection string    
    conn = psycopg2.connect(host=ENDPOINT, port=PORT, user=USER, 
        database=DATABASE, password=PASSWORD, sslmode='prefer', 
        sslrootcert="[full path]rds-combined-ca-bundle.pem")
    return conn


def fetch_results(sql, conn):
    cur = conn.cursor()
    cur.execute(sql)
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(cur.fetchall(), columns=columns)
    cur.close()
    return df


def exec_sql(sql, conn, commit_changes=False):
    cur = conn.cursor()
    cur.execute(sql)
    if commit_changes:
        conn.commit() # <- We MUST commit to reflect the inserted data
    cur.close()
    return "Success :)"

In [3]:
# create connection
secrets = get_secret()
conn = create_rw_conn(secrets=secrets)

## Load new data

In [4]:
new_weather_data = pd.read_csv("data/new-weather-data.csv", dtype={'fips_code':str})
new_weather_data

,fips_code,county_name,state,temperature
0,06075,San Francisco,CA,53.7
1,48235,Irion,TX,84.3
2,48309,McLennan,TX,86.3
3,48173,Glasscock,TX,83.4
4,48401,Rusk,TX,84.3
...,...,...,...,...
60,06111,Ventura,CA,75.2
61,48295,Lipscomb,TX,65.4
62,06029,Kern,CA,83.2
63,06053,Monterey,CA,74.8


In [5]:
new_weather_data['state'].value_counts()

TX    39
CA    13
NY    13
Name: state, dtype: int64

In [6]:
# remap state
new_weather_data['state'] = new_weather_data['state'].map({'CA': 'california', 'TX': 'texas', 'NY': 'newyork'})
new_weather_data.head()

,fips_code,county_name,state,temperature
0,06075,San Francisco,california,53.7
1,48235,Irion,texas,84.3
2,48309,McLennan,texas,86.3
3,48173,Glasscock,texas,83.4
4,48401,Rusk,texas,84.3


In [7]:
# new_weather_data['state'] = new_weather_data['state'].map({'california': 'CA', 'texas': 'TX', 'newyork': 'NY'})

### Create new schemas and tables

In [8]:
sql_create_schema = """
    CREATE SCHEMA IF NOT EXISTS {state};
    """

In [9]:
sql_create_table = """
    CREATE TABLE IF NOT EXISTS {state}.weather_county (
                id serial PRIMARY KEY,
                fips_code varchar(10) UNIQUE,
                county_name varchar(20),
                temperature numeric NOT NULL,
                fahrenheit bool DEFAULT true,
                updated_at timestamp default now(),
                created_at timestamp default now()
            );
    """

In [10]:
grouped = new_weather_data.groupby(by='state')

In [11]:
for state, df in grouped:
    # don't forget to commit changes!!
    exec_sql(sql_create_schema.format(state=state), conn=conn, commit_changes=True)
    exec_sql(sql_create_table.format(state=state), conn=conn, commit_changes=True)
    print("[{}] schema + table created".format(state))

[california] schema + table created
[newyork] schema + table created
[texas] schema + table created


### Check if new schemas/tables were successfully created

In [12]:
sql = """
    SELECT s.catalog_name, s.schema_owner, s.schema_name, t.table_name
    FROM information_schema.schemata s
    JOIN information_schema.tables t ON s.catalog_name = t.table_catalog and s.schema_name = t.table_schema
    WHERE schema_owner = 'johnnyboycurtis';
    """

fetch_results(sql=sql, conn=conn)

,catalog_name,schema_owner,schema_name,table_name
0,analytics,johnnyboycurtis,texas,weather_county
1,analytics,johnnyboycurtis,california,weather_county
2,analytics,johnnyboycurtis,newyork,weather_county


In [13]:
# close connection to prevent conflicts
conn.close()

### Insert new data/update existing data

In [14]:
from psycopg2.extras import execute_values

In [15]:
# new connection
conn = create_rw_conn(secrets=secrets)

In [16]:
sql_insert = """    
    INSERT INTO {state}.weather_county(fips_code, county_name, temperature)
    VALUES %s
    ON CONFLICT (fips_code) DO UPDATE
    SET
        temperature=excluded.temperature,
        updated_at=NOW()
    ;
    """

In [17]:
# test
#new_weather_data = new_weather_data.loc[new_weather_data['state'] == 'texas']
#new_weather_data

In [18]:
grouped = new_weather_data.groupby(by='state')

conn = create_rw_conn(secrets=secrets)

In [19]:
for state, df in grouped:
    # select only the neccessary columns
    df = df[['fips_code', 'county_name', 'temperature']]
    print("[{}] upsert...".format(state))
    # convert dataframe into list of lists for `execute_values`
    data = [tuple(x) for x in df.values.tolist()]
    cur = conn.cursor()
    execute_values(cur, sql_insert.format(state=state), data)
    conn.commit() # <- We MUST commit to reflect the inserted data
    print("[{}] changes were commited...".format(state))
    cur.close()

[california] upsert...
[california] changes were commited...
[newyork] upsert...
[newyork] changes were commited...
[texas] upsert...
[texas] changes were commited...


### Review data

In [20]:
sql = "SELECT * FROM california.weather_county;"
df = fetch_results(sql=sql, conn=conn)
df.head(10)

,id,fips_code,county_name,temperature,fahrenheit,updated_at,created_at
0,3,06075,San Francisco,53.7,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
1,4,06071,San Bernardino,82.0,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
2,5,06107,Tulare,72.7,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
3,6,06001,Alameda,66.2,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
4,7,06035,Lassen,67.8,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
5,8,06113,Yolo,83.2,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
6,9,06063,Plumas,70.8,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
7,10,06073,San Diego,76.7,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
8,11,06059,Orange,73.2,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
9,12,06111,Ventura,75.2,True,2022-11-25 22:54:28.716321,2022-11-25 21:26:01.829159
